<a href="https://colab.research.google.com/github/rj-adity/Aditya_Bright_Automative_company_analysis_CS4N/blob/main/cervical_cancer2502.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rjaditya2","key":"93849502209a84c3350e62495ef5936e"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed


Dataset URL: https://www.kaggle.com/datasets/prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed
License(s): other
^C


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mv cervical-cancer-largest-dataset-sipakmed.zip /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mv: cannot stat 'cervical-cancer-largest-dataset-sipakmed.zip': No such file or directory


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import zipfile

zip_path="/content/drive/MyDrive/cervical-cancer-largest-dataset-sipakmed.zip"
extract_path = "dataset"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

!ls dataset

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/cervical-cancer-largest-dataset-sipakmed.zip'

In [ ]:
import os
dataset_Directory="dataset"

for root, dirs, files in os.walk(dataset_Directory):
  print(f"Root: {root}")
  print(f"Directories: {dirs}")
  print(f"Files: {files}")


In [ ]:
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
base_dir = base_dir = "/content/dataset"
categories = ["im_Dyskeratotic", "im_Koilocytotic", "im_Metaplastic",
              "im_Parabasal", "im_Superficial-Intermediate"]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(style="whitegrid")

Counting variety of images avilable in our datasets.


In [ ]:
image_counts = {}

for category in categories:
    category_path = os.path.join(base_dir, category, category, "CROPPED")
    image_counts[category] = len([f for f in os.listdir(category_path) if f.endswith(('.bmp', '.jpg', '.png'))])

print(image_counts)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=list(image_counts.keys()), y=list(image_counts.values()), palette='viridis')
plt.title('Count of Images in Each Category')
plt.xlabel('Categories')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(image_counts.values(), labels=image_counts.keys(), autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Images by Category')
plt.axis('equal')
plt.show()

In [ ]:
import cv2

Displaying variety of images in diffrent conditions

In [ ]:
def display_images_from_folders(base_dir, categories, num_images=5):
    plt.figure(figsize=(15, 10))

    for i, category in enumerate(categories):
        category_path = os.path.join(base_dir, category, category, "CROPPED")
        image_files = [f for f in os.listdir(category_path) if f.endswith(('.bmp', '.jpg', '.png'))]

        for j, image_file in enumerate(image_files[:num_images]):

            img_path = os.path.join(category_path, image_file)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            plt.subplot(len(categories), num_images, i * num_images + j + 1)
            plt.imshow(image)
            plt.axis('off')
            plt.title(f"{category}")

    plt.tight_layout()
    plt.show()

In [ ]:
display_images_from_folders(base_dir, categories, num_images=5)

Making path for better storing of image

In [ ]:
def create_image_paths_dataframe(base_dir, categories):
    data = []

    for category in categories:
        category_path = os.path.join(base_dir, category, category, "CROPPED")

        image_files = [f for f in os.listdir(category_path) if f.endswith(('.bmp', '.jpg', '.png'))]

        for image_file in image_files:
            img_path = os.path.join(category_path, image_file)
            data.append({'image_path': img_path, 'label': category})

    df_image_paths = pd.DataFrame(data)

    return df_image_paths

In [ ]:
df_image_paths = create_image_paths_dataframe(base_dir, categories)
df_image_paths.head()

In [ ]:
df_image_paths.tail()

In [ ]:
df_image_paths['label'].unique()

In [ ]:
df_image_paths['label'].value_counts()

Handling imbalances of our datsets

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(df_image_paths[['image_path']], df_image_paths['label'])

In [ ]:
df_resampled = pd.DataFrame(X_resampled, columns=['image_path'])
df_resampled['label'] = y_resampled


In [ ]:
print("\nClass distribution after oversampling:")
print(df_resampled['label'].value_counts())

In [ ]:
import time
import shutil
import pathlib
import itertools
from PIL import Image

import cv2
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_resampled.info()

In [ ]:
train_df_new, temp_df_new = train_test_split(
    df_resampled,
    train_size=0.8,
    shuffle=True,
    random_state=42,
    stratify=df_resampled['label']
)

valid_df_new, test_df_new = train_test_split(
    temp_df_new,
    test_size=0.5,
    shuffle=True,
    random_state=42,
    stratify=temp_df_new['label']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 16
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

tr_gen = ImageDataGenerator(rescale=1./255)
ts_gen = ImageDataGenerator(rescale=1./255)

train_gen_new = tr_gen.flow_from_dataframe(
    train_df_new,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    class_mode='sparse',
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size
)

valid_gen_new = ts_gen.flow_from_dataframe(
    valid_df_new,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    class_mode='sparse',
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size
)

test_gen_new = ts_gen.flow_from_dataframe(
    test_df_new,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    class_mode='sparse',
    color_mode='rgb',
    shuffle=False,
    batch_size=batch_size
)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("Using GPU")
else:
    print("Using CPU")

In [ ]:
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

input_shape = (224, 224, 3)
num_classes = len(train_gen_new.class_indices)

cnn_model = create_cnn_model(input_shape, num_classes)

cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = cnn_model.fit(
    train_gen_new,
    validation_data=valid_gen_new,
    epochs=10,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
test_labels = test_gen_new.classes
predictions = cnn_model.predict(test_gen_new)
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
report = classification_report(test_labels, predicted_classes, target_names=list(test_gen_new.class_indices.keys()))
print(report)

In [ ]:
conf_matrix = confusion_matrix(test_labels, predicted_classes)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=list(test_gen_new.class_indices.keys()), yticklabels=list(test_gen_new.class_indices.keys()))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Provided confusion matrix
confusion_matrix = np.array([
    [72, 10, 0, 0, 1],
    [2, 70, 9, 1, 1],
    [3, 0, 73, 4, 3],
    [0, 0, 4, 71, 9],
    [0, 5, 5, 3, 70]
])

# Calculate precision for each category
true_positives = np.diag(confusion_matrix)
false_positives = np.sum(confusion_matrix, axis=0) - true_positives
precision = true_positives / (true_positives + false_positives)

# Updated precision values for "Xception with Attention" as an example
data = {
    'Category': [
        'im_Dyskeratotic',
        'im_Koilocytotic',
        'im_Metaplastic',
        'im_Parabasal',
        'im_Superficial-Intermediate'
    ],
    'Simple CNN': [0.95, 0.86, 0.79, 0.90, 0.93],
    'InceptionV3': [0.89, 0.62, 0.85, 0.89, 0.85],
    'Xception': [0.94, 0.83, 0.87, 0.96, 0.98],
    'Xception with Attention': precision.round(2),  # Updated values
    'Inception with Attention': [0.86, 0.72, 0.92, 0.78, 0.85]
}

df = pd.DataFrame(data)
df.set_index('Category', inplace=True)

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.18
x = np.arange(len(df.index))

for i, model in enumerate(df.columns):
    bars = ax.bar(x + i * bar_width, df[model], width=bar_width, label=model)
    for bar in bars:
        yval = bar.get_height()
        ax.annotate(f'{yval:.2f}', xy=(bar.get_x() + bar.get_width() / 2, yval),
                    ha='center', va='bottom', fontsize=10)

ax.set_xlabel('Categories', fontsize=14)
ax.set_ylabel('Precision', fontsize=14)
ax.set_title('Comparative Precision of Different Models', fontsize=16)
ax.set_xticks(x + bar_width * 2)
ax.set_xticklabels(df.index, fontsize=12)
ax.legend(title='Models', fontsize=12, loc='lower right')

plt.tight_layout()
plt.show()


In [ ]:
cnn_model.save('cancer_classifier.keras')

In [ ]:
from google.colab import files
files.download('cancer_classifier.keras')


In [ ]:
with zipfile.ZipFile('cancer_classifier.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('cancer_classifier.keras')

# Verify the zip file has been created
os.listdir()  # List the files to check

In [ ]:
import shutil

# Move the zip file to Google Drive (change path if necessary)
shutil.move('cancer_classifier.zip', '/content/drive/My Drive/cancer_classifier.zip')
